# Metodo de Newton para solucionar el Solver de Markovitz

In [0]:
import numpy as np
import cupy as cp
import solver.extraer_datos_yahoo as extrae
import solver.funciones_auxiliares as aux
import solver.line_search as line
import solver.modelo_markowitz as mkv
import solver.utils as utils
import solver.optimizacion_numerica as opt
#from utils import inc_index, dec_index, compute_error, norm_residual, condicion_cupy
#from line_search import line_search_by_backtracking, line_search_for_residual_by_backtracking

## Cargamos la información y calculamos los parámetros

Primero cargamos los precios de las acciones que se han escogido para esta implementación

In [0]:
stocks = ['COP','AMT','LIN','LMT','AMZN','WMT','JNJ','VTI','MSFT','GOOG','XOM','CCI','BHP.AX','UNP',
'BABA','NSRGY','RHHBY','VOO','AAPL','FB','CVX','PLD','RIO.L','HON','HD','PG','UNH','BRK-A','V','0700.HK',
'RDSA.AS','0688.HK','AI.PA','RTX','MC.PA','KO','PFE','JPM','005930.KS','VZ','RELIANCE.NS','DLR','2010.SR',
'UPS','7203.T','PEP','MRK','1398.HK','MA','T']

In [3]:
datos = extrae.extraer_datos_yahoo(stocks)

[*********************100%***********************]  50 of 50 downloaded


Usando las fucniones auxiliares definidas previamente calculamos los rendimientos esperados y la matriz de varianza y covarianza para las acciones

In [0]:
mu = aux.calcular_rendimiento(datos)

In [0]:
S = aux.calcular_varianza(datos)

Calcularemos cuál fue el rendimiento máximo obtenido (de entre las medias de los datos históricos) y lo usaremos como el rendimiento deseado en el problema de optimización

In [6]:
r=max(mu).item()
r

0.4022108787760788

## Resolvemos con el Método de Newton usando diferencias finitas

La función a optimizar será la siguiente:

In [0]:
fo = lambda w: w@S@w

Usamos la solución analítica para tener la solución con la cual comparar.

In [0]:
w_ast = mkv.markowitz(r,mu,S)

La matriz $A$ de restricciones está dada por:

In [9]:
n = mu.shape[0]
A = cp.concatenate((mu,cp.ones(n))).reshape(2,n)
A

array([[ 0.13254293,  0.02782955,  0.25778345, -0.01787111, -0.02008539,
        -0.02286483,  0.19078421,  0.05743478,  0.17062275,  0.40221088,
         0.13618598,  0.02364206,  0.0476638 ,  0.13217291, -0.09666644,
        -0.03426582,  0.15880965,  0.17870289,  0.18557566,  0.15042008,
         0.08379133,  0.07154137,  0.08843336,  0.02202303,  0.07353272,
         0.13731909,  0.23605478,  0.19942876,  0.06806557,  0.26292508,
         0.07357148,  0.06803817,  0.03867589,  0.05090149,  0.14401416,
        -0.09339291,  0.23072551,  0.05036842,  0.05587177, -0.01264577,
        -0.01474117,  0.20678446,  0.06274014, -0.02729424,  0.1990038 ,
         0.07054765,  0.06563628,  0.04203765,  0.0717407 , -0.13227261],
       [ 1.        ,  1.        ,  1.        ,  1.        ,  1.        ,
         1.        ,  1.        ,  1.        ,  1.        ,  1.        ,
         1.        ,  1.        ,  1.        ,  1.        ,  1.        ,
         1.        ,  1.        ,  1.        ,  1.

Por otro lado el vector $b$ está dado por:

In [10]:
b = cp.array([r,1])
b

array([0.40221088, 1.        ])

Definimos el punto inicial con la función definida anteriormente:

In [0]:
w_0 = utils.feasible_markowitz(r,mu)

Definimos los parámetros para el método de Newton de la siguiente manera:

In [0]:
tol=1e-8
tol_backtracking=1e-14
p_ast=fo(w_ast)
maxiter=50

In [14]:
[w,total_of_iterations,Err_plot,x_plot]=opt.Newtons_method_feasible_init_point(fo,A, w_0,tol, tol_backtracking, w_ast, p_ast, maxiter)



I	Normgf 	Newton Decrement	Error x_ast	Error p_ast	line search	CondHf
0	0.0019	0.0009	1.1003	4.6964	---		10050.1996
1	0.0019	0.0	0.0424	0.0002	1	10050.1996
2	0.0019	0.0	0.003	0.0	1	10050.1996
Error of x with respect to x_ast: 0.003025598335118704
Approximate solution: [ 1.58428546e-01 -2.08616309e-02  1.58020949e-01 -9.28935082e-02
  3.16991935e-02  5.10945480e-02  8.72922956e-02  1.26346166e-02
  3.73103284e-02  2.86016263e-01 -5.94665541e-03  2.23126418e-03
  2.03110573e-01  9.45805956e-02  2.28799709e-02  1.50331910e-02
  7.62589108e-03  2.97057955e-02  5.77183696e-02  1.98175883e-01
  1.19874923e-01  1.27950342e-01  1.41431816e-01  1.36290060e-02
  8.84466664e-02  1.50547999e-01  1.69210858e-01  7.72068032e-02
  8.09578029e-02  8.25608723e-02  1.92191572e-01 -2.38661332e-02
  2.64704653e-02  7.70401324e-02  2.20865617e-02 -1.08224222e-01
  1.64026211e-01  1.77103658e-02  6.16445527e-02 -1.08192451e-01
 -5.03004393e-02  1.38878495e-01  1.03603985e-01 -4.27834232e-02
  1.08153614e-02

La solución dada por el Método de Newton usando las diferenciación numérica es muy similar a la del método cerrado. Siendo el error de:

In [16]:
Err_plot[-1]

3.520979637982383e-07

In [15]:
w

array([ 1.58428546e-01, -2.08616309e-02,  1.58020949e-01, -9.28935082e-02,
        3.16991935e-02,  5.10945480e-02,  8.72922956e-02,  1.26346166e-02,
        3.73103284e-02,  2.86016263e-01, -5.94665541e-03,  2.23126418e-03,
        2.03110573e-01,  9.45805956e-02,  2.28799709e-02,  1.50331910e-02,
        7.62589108e-03,  2.97057955e-02,  5.77183696e-02,  1.98175883e-01,
        1.19874923e-01,  1.27950342e-01,  1.41431816e-01,  1.36290060e-02,
        8.84466664e-02,  1.50547999e-01,  1.69210858e-01,  7.72068032e-02,
        8.09578029e-02,  8.25608723e-02,  1.92191572e-01, -2.38661332e-02,
        2.64704653e-02,  7.70401324e-02,  2.20865617e-02, -1.08224222e-01,
        1.64026211e-01,  1.77103658e-02,  6.16445527e-02, -1.08192451e-01,
       -5.03004393e-02,  1.38878495e-01,  1.03603985e-01, -4.27834232e-02,
        1.08153614e-02, -2.33911126e+00,  1.94282246e-01,  2.63290259e-01,
        1.20150533e-01, -1.05386379e-01])

In [16]:
w_ast

array([ 1.58450459e-01, -2.08710279e-02,  1.58051613e-01, -9.28970852e-02,
        3.17093838e-02,  5.10824537e-02,  8.71800626e-02,  1.26389547e-02,
        3.72861469e-02,  2.86000507e-01, -5.98810147e-03,  2.24206232e-03,
        2.03075836e-01,  9.46030741e-02,  2.28766788e-02,  1.49919976e-02,
        7.60706433e-03,  2.96676402e-02,  5.76521006e-02,  1.98109863e-01,
        1.19928144e-01,  1.27869501e-01,  1.41300419e-01,  1.36285005e-02,
        8.83904753e-02,  1.50479914e-01,  1.69293512e-01,  7.72037012e-02,
        8.09121352e-02,  8.24658724e-02,  1.92197879e-01, -2.40095431e-02,
        2.64375219e-02,  7.69647088e-02,  2.20741648e-02, -1.08207562e-01,
        1.64032748e-01,  1.77020932e-02,  6.16398087e-02, -1.08210745e-01,
       -5.03462807e-02,  1.38834320e-01,  1.03567400e-01, -4.28198353e-02,
        1.06872745e-02, -2.33314798e+00,  1.89811544e-01,  2.63296457e-01,
        1.20089773e-01, -1.05535600e-01])

La varianza entre ambos métodos también es muy similar

In [19]:
w@S@w

array(9.45979579e-05)

In [20]:
w_ast@S@w_ast

array(9.45979246e-05)

In [21]:
utils.compute_error(w@S@w,w_ast@S@w_ast)

array(3.5209784e-07)

Además cumple con la restricción del rendimiento y la suma de pesos igual a 1

In [24]:
w@mu

array(0.40221088)

In [25]:
utils.compute_error(w@mu, r)

array(1.38015042e-16)

In [26]:
sum(x)

array(1.)

## Resolvemos con el Método de Newton usando funciones simbólicas

Ahora utilizaremos de nuevo el método de Newton usando las funciones simbólicas:

In [0]:
import inspect

In [30]:
lines = inspect.getsource(opt.gfo_cp_mark)
print(lines)

def gfo_cp_mark(x,Sigma):
    first_block = Sigma@x
    #es necesario pasar los bloques a numpy para poder hacerles concatenate 
    return first_block



In [31]:
lines = inspect.getsource(opt.Hfo_cp_mark)
print(lines)

def Hfo_cp_mark(x, Sigma):
    first_block = Sigma 
    return first_block



In [32]:
[w,total_of_iterations,Err_plot,x_plot]=opt.Newtons_method_feasible_init_point(fo,A, w_0,tol, tol_backtracking, w_ast, p_ast, maxiter, opt.gfo_cp_mark, opt.Hfo_cp_mark,S)

I	Normgf 	Newton Decrement	Error x_ast	Error p_ast	line search	CondHf
0	0.001	0.0004	1.1003	4.6964	---		10080.7985
1	0.001	0.0	0.0	0.0	1	10080.7985
Error of x with respect to x_ast: 6.406978892882662e-12
Approximate solution: [ 1.58450459e-01 -2.08710279e-02  1.58051613e-01 -9.28970852e-02
  3.17093838e-02  5.10824537e-02  8.71800626e-02  1.26389547e-02
  3.72861469e-02  2.86000507e-01 -5.98810147e-03  2.24206232e-03
  2.03075836e-01  9.46030741e-02  2.28766788e-02  1.49919976e-02
  7.60706433e-03  2.96676402e-02  5.76521006e-02  1.98109863e-01
  1.19928144e-01  1.27869501e-01  1.41300419e-01  1.36285005e-02
  8.83904753e-02  1.50479914e-01  1.69293512e-01  7.72037012e-02
  8.09121352e-02  8.24658724e-02  1.92197879e-01 -2.40095431e-02
  2.64375219e-02  7.69647088e-02  2.20741648e-02 -1.08207562e-01
  1.64032748e-01  1.77020932e-02  6.16398087e-02 -1.08210745e-01
 -5.03462807e-02  1.38834320e-01  1.03567400e-01 -4.28198353e-02
  1.06872745e-02 -2.33314798e+00  1.89811544e-01  2.6329645

La solución dada por el Método de Newton usando las funciones simbólicas es prácticamente igual al método cerrado método cerrado. Siendo el error de:

In [33]:
Err_plot[-1]

4.011406825379125e-15

In [34]:
w

array([ 1.58450459e-01, -2.08710279e-02,  1.58051613e-01, -9.28970852e-02,
        3.17093838e-02,  5.10824537e-02,  8.71800626e-02,  1.26389547e-02,
        3.72861469e-02,  2.86000507e-01, -5.98810147e-03,  2.24206232e-03,
        2.03075836e-01,  9.46030741e-02,  2.28766788e-02,  1.49919976e-02,
        7.60706433e-03,  2.96676402e-02,  5.76521006e-02,  1.98109863e-01,
        1.19928144e-01,  1.27869501e-01,  1.41300419e-01,  1.36285005e-02,
        8.83904753e-02,  1.50479914e-01,  1.69293512e-01,  7.72037012e-02,
        8.09121352e-02,  8.24658724e-02,  1.92197879e-01, -2.40095431e-02,
        2.64375219e-02,  7.69647088e-02,  2.20741648e-02, -1.08207562e-01,
        1.64032748e-01,  1.77020932e-02,  6.16398087e-02, -1.08210745e-01,
       -5.03462807e-02,  1.38834320e-01,  1.03567400e-01, -4.28198353e-02,
        1.06872745e-02, -2.33314798e+00,  1.89811544e-01,  2.63296457e-01,
        1.20089773e-01, -1.05535600e-01])

In [35]:
w_ast

array([ 1.58450459e-01, -2.08710279e-02,  1.58051613e-01, -9.28970852e-02,
        3.17093838e-02,  5.10824537e-02,  8.71800626e-02,  1.26389547e-02,
        3.72861469e-02,  2.86000507e-01, -5.98810147e-03,  2.24206232e-03,
        2.03075836e-01,  9.46030741e-02,  2.28766788e-02,  1.49919976e-02,
        7.60706433e-03,  2.96676402e-02,  5.76521006e-02,  1.98109863e-01,
        1.19928144e-01,  1.27869501e-01,  1.41300419e-01,  1.36285005e-02,
        8.83904753e-02,  1.50479914e-01,  1.69293512e-01,  7.72037012e-02,
        8.09121352e-02,  8.24658724e-02,  1.92197879e-01, -2.40095431e-02,
        2.64375219e-02,  7.69647088e-02,  2.20741648e-02, -1.08207562e-01,
        1.64032748e-01,  1.77020932e-02,  6.16398087e-02, -1.08210745e-01,
       -5.03462807e-02,  1.38834320e-01,  1.03567400e-01, -4.28198353e-02,
        1.06872745e-02, -2.33314798e+00,  1.89811544e-01,  2.63296457e-01,
        1.20089773e-01, -1.05535600e-01])

La varianza entre ambos métodos también es prácticamente la misma

In [36]:
w@S@w

array(9.45979246e-05)

In [37]:
w_ast@S@w_ast

array(9.45979246e-05)

In [38]:
utils.compute_error(w@S@w,w_ast@S@w_ast)

array(4.01140683e-15)

Y de igual manera cumple con la restricción del rendimiento y la suma de pesos igual a 1

In [39]:
w@mu

array(0.40221088)

In [40]:
utils.compute_error(w@mu, r)

array(0.)

In [41]:
sum(x)

array(1.)